In [7]:
import pandas as pd
import os
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"
output_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2\Procesados"

# Obtener lista de archivos .xlsx en el directorio de salida
xlsx_files = [f for f in os.listdir(output_dir) if f.endswith('.xlsx')]

# Asegurarse de que el directorio de salida existe
os.makedirs(output_dir, exist_ok=True)

# Imprimir los nombres de las hojas y las columnas de cada archivo .xlsx en el directorio de salida
for file in xlsx_files:
    file_path = os.path.join(output_dir, file)
    print(f"Archivos en {file_path}:")
    
    excel_file = pd.ExcelFile(file_path)
    print(f"Nombres de las hojas: {excel_file.sheet_names}")

    for sheet_name in excel_file.sheet_names:
        df = excel_file.parse(sheet_name)
        print(f"Columnas de la hoja '{sheet_name}': {df.columns.tolist()}")
    
    print('-' * 50)  # Separador entre archivos


Archivos en C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2\Procesados\procesado_20250421_221206_Internet.xlsx:
Nombres de las hojas: ['Acc_vel_loc_sinrangos', 'Velocidad_sin_Rangos', 'Accesos_tecnologia_localidad', 'Velocidad % por prov', 'Totales VMD', 'Totales Accesos Por Tecnología', 'Accesos Por Tecnología', 'Penetración-poblacion', 'Penetracion-hogares', 'Penetracion-totales', 'Totales Accesos por rango', 'Accesos por rangos', 'Dial-BAf', 'Totales Dial-BAf', 'Ingresos ']
Columnas de la hoja 'Acc_vel_loc_sinrangos': ['Partido', 'Localidad', 'link Indec', 'Velocidad (Mbps)', 'Provincia', 'Accesos']
Columnas de la hoja 'Velocidad_sin_Rangos': ['Año', 'Trimestre', 'Provincia', 'Velocidad', 'Accesos', 'Fecha_Trimestre']
Columnas de la hoja 'Accesos_tecnologia_localidad': ['Provincia', 'Partido', 'Localidad', 'Tecnologia', 'Link Indec', 'Accesos']
Columnas de la hoja 'Velocidad % por prov': ['Año', 'Trimestre', 'Provincia', 'Mbps (Media de bajada)', 'Fecha_Trimestre']
Colum

In [7]:
import pandas as pd
import os
from datetime import datetime

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"
output_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2\Procesados"

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

xlsx_files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx') and "(1)" not in f]

# Función para limpiar la columna Trimestre
def limpiar_trimestre(valor):
    if isinstance(valor, str):
        valor = valor.replace('*', '')  # Eliminar asteriscos
    try:
        # Convertir a entero y verificar que esté en el rango de trimestres
        trimestre = int(valor)
        if trimestre in [1, 2, 3, 4]:  # Asegurarse de que esté en el rango 1-4
            return trimestre
    except ValueError:
        pass 
    return 0 

# Función para convertir la columna Año a tipo datetime, con un límite de años
def convertir_a_fecha(valor):
    if pd.notna(valor):  # Verificar que el valor no sea nulo
        try:
            # Intentar convertir el valor a entero (representando el año)
            año = int(valor)
    
            if 2010 <= año <= datetime.now().year:
                return datetime(año, 1, 1)  # Crear una fecha con el año y mes/día por defecto
            else:
                return 0  # Si el año no está dentro del rango, devolver 0
        except (ValueError, TypeError):
            pass  
    return 0 

# Función para generar fecha basada en el Trimestre
def generar_fecha_trimestre(año, trimestre):
    try:
        # Asegurarse de que trimestre sea un entero
        trimestre = int(trimestre)  
        if pd.notna(año) and isinstance(año, (int, float)) and año != 0:
            # Verificar que el año es un entero
            año = int(año)
            if trimestre == 1:
                return datetime(año, 1, 1) 
            elif trimestre == 2:
                return datetime(año, 4, 1)  
            elif trimestre == 3:
                return datetime(año, 7, 1)  
            elif trimestre == 4:
                return datetime(año, 10, 1)  
    except (ValueError, TypeError):
        pass  
    return 0  

# Función para generar fecha basada en el Mes
def generar_fecha_mes(año, mes):
    try:
        mes = int(mes) 
        return datetime(año, mes, 1)  # Usar el primer día del mes
    except (ValueError, TypeError):
        return 0  # Si no se puede convertir, devolver 0

# Función para limpiar las filas con datos inconsistentes
def limpiar_filas(df):
    # Limpiar la columna 'Año' asegurándonos de que los valores sean enteros
    if 'Año' in df.columns:
        df['Año'] = pd.to_numeric(df['Año'], errors='coerce')  # Convertir a numérico, reemplaza los no convertibles por NaN
        df = df[df['Año'].apply(lambda x: 2010 <= x <= datetime.now().year if pd.notna(x) else False)]  # Filtrar los años válidos
    
    # Limpiar la columna 'Trimestre'
    if 'Trimestre' in df.columns:
        df = df[df['Trimestre'].apply(lambda x: x in [1, 2, 3, 4] if pd.notna(x) else False)]  # Filtrar trimestres válidos
    
    return df

# Procesar cada archivo
for file in xlsx_files:
    file_path = os.path.join(input_dir, file)
    df_sheets = pd.read_excel(file_path, sheet_name=None)
    processed_sheets = {}

    for sheet_name, df in df_sheets.items():
        # Eliminar la hoja "Totales Dial-BAf" si existe
        if sheet_name == "Totales Dial-BAf":
            continue  # Salta este sheet y no lo procesa
        
        # Eliminar filas que contengan "Sin Datos" en cualquier columna
        df = df[~df.apply(lambda row: row.astype(str).str.contains("Sin Datos").any(), axis=1)]

        # Rellenar valores nulos con 0
        df = df.fillna(0)

        # Eliminar columnas con más del 50% de valores nulos
        df = df.drop(columns=df.columns[df.isnull().mean() > 0.5])

        # Eliminar columnas con nombres específicos como 'Correo oficial + Correos Privados (en Pesos)' o 'Unnamed: x'
        df = df.drop(columns=[col for col in df.columns if 'Correo oficial + Correos Privados (en Pesos)' in col or 'Unnamed:' in col])

        # Eliminar las columnas que contengan 'otros'
        df = df.drop(columns=[col for col in df.columns if 'otros' in col.lower()])

        # Limpiar las filas con valores inconsistentes o erróneos
        df = limpiar_filas(df)

        if 'Trimestre' in df.columns:
            df['Trimestre'] = df['Trimestre'].apply(limpiar_trimestre)  # Limpiar los trimestres

        if 'Año' in df.columns:
            df['Año'] = df['Año'].apply(convertir_a_fecha)  # Limitar y convertir el año a fecha

        if 'Mes' in df.columns:
            df['Fecha'] = df.apply(lambda row: generar_fecha_mes(row['Año'], row['Mes']) if pd.notna(row['Año']) and pd.notna(row['Mes']) else 0, axis=1)
        
        if 'Trimestre' in df.columns:
            df['Fecha_Trimestre'] = df.apply(lambda row: generar_fecha_trimestre(row['Año'], row['Trimestre']) if pd.notna(row['Año']) and pd.notna(row['Trimestre']) else 0, axis=1)

        # Eliminar la columna Fecha_Trimestre
        if 'Fecha_Trimestre' in df.columns:
            df = df.drop(columns=['Fecha_Trimestre'])

        # Agregar el DataFrame procesado al diccionario
        processed_sheets[sheet_name] = df

    # Crear un timestamp único para evitar sobrescribir archivos
    processed_file_path = os.path.join(output_dir, f"procesado_{file}")

    # Guardar el archivo procesado sin el nombre con números
    with pd.ExcelWriter(processed_file_path, engine='xlsxwriter') as writer:
        for sheet_name, processed_df in processed_sheets.items():
            processed_df.to_excel(writer, sheet_name=sheet_name, index=False)




